# Objectives
* Gain hands on experience working with pyConText and its resources
* Understand and develop Targets
* Understand and develop Modifiers
* Graph and visualize Targets and Modifiers together
* Gain tools for group project in reducing False Negatives and False Positives (F1 measure)

In [3]:
#You need to download an nltk model if you haven't
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/jianlins/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
import os
# we will definitely need pyConText
import pyConTextNLP
from pyConTextNLP import pyConText
from pyConTextNLP import itemData
import urllib
from radnlp.data import classrslts 
from pyConTextNLP.display.html import mark_document_with_html
print(pyConTextNLP.__version__)
# useful utilities in RadNLP as well

import pandas as pd
# packages for interaction
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML, Image
import ipywidgets
# and also our utilities for this class
from nlp_pneumonia_utils import Annotation
from nlp_pneumonia_utils import AnnotatedDocument
from nlp_pneumonia_utils import read_brat_annotations
from nlp_pneumonia_utils import read_doc_annotations
from nlp_pneumonia_utils import read_annotations
from nlp_pneumonia_utils import calculate_prediction_metrics
from nlp_pneumonia_utils import mark_text
from nlp_pneumonia_utils import clearPyConTextRegularExpressions
from nlp_pneumonia_utils import pneumonia_annotation_html_markup
from nlp_pneumonia_utils import mark_document_with_html
from nlp_pneumonia_utils import view_single_sentence_graph
from nlp_pneumonia_utils import markup_sentence

from pycontext_quiz import identify_target_category
from pycontext_quiz import file_delimiter_quiz
from pycontext_quiz import modifier_directionality_quiz
from pycontext_quiz import second_most_frequent_modifier_quiz

from visual import Vis
from visual import snippets_markup
from visual import view_pycontext_output

from textblob import TextBlob

print('Imported pneumonia nlp utilities...')

0.7.0.0
Imported pneumonia nlp utilities...


# First thing, let's load our training set

In [2]:
annotated_doc_map = read_doc_annotations('data/training_v2.zip')

# let's also use a simple list of documents as well as this map
annotated_docs = list(annotated_doc_map.values())

print('Total Annotated Documents : {0}'.format(len(annotated_docs)))

Reading annotations from file : data/training_v2.zip
Opening local file : data/training_v2.zip
Total Annotated Documents : 70


In [3]:
# let us set up an example document to work with
example_document = """
PORTABLE CHEST:  Comparison made to prior film from X:XX a.m. the same day.
     
The ET tube and nasogastric tube remain in good position. Cardiac and
mediastinal contours are stable. No acute changes are seen within the lung
parenchyma; specifically, there is no evidence of new infiltrate (skin folds
do project over the right lung). No consolidation on either side.

IMPRESSION: No evidence of pneumonia."""

example_sentence = """IMPRESSION: No evidence of pneumonia."""

## Our first goal is to convert the "keywords" that we worked on in our previous activity into pyConText "targets" where each target can have:
* Standard form
* Category 
* Additional lexical variants (with regular expressions)

## Before we continue, note that any itemData in pyConText has 4 parts:
1. The literal (e.g. "pneumonia", "pneumoniathorax", "can rule out", "cannot be excluded", etc)
2. The category (e.g. "EVIDENCE_OF_PNEUMONIA")
3. The regular expression (optional) used to capture the literal in the text. If no regular expression is provided, a regular expression is generated literally from the literal.
4. The rule (optional). If the itemData is being used as a modifier, the rule states what direction the modifier operates in the sentence: current valid values are: "forward", the item can modify objects following it in the sentence; "backward", the item can modify objects preceding it in the sentence; or "bidirectional", the item can modify objects preceding and following it in the sentence.

## Additionally, itemData can be instantiated in code or from a file.  Let's start with code:

In [4]:
# Now let's set up some rules for pyConText for EVIDENCE_OF_PNEUMONIA
# At this moment, we will just set up these "concepts" and well handle modifiers for them after that

targets1 = []
modifiers1 = []

# so before we add targets, remember from above that they will look like this : 
# targets = itemData(["literal", "CATEGORY", "regular expression(s)", "empty or forward or backward or bidirectional"])

# so now let's set this up for "pneumonia" with the category "EVIDENCE_OF_PNEUMONIA"
targets1 = itemData.itemData(["pneumonia", "EVIDENCE_OF_PNEUMONIA", "", ""])

# let's go ahead and use this now on one single example sentence:
markup = markup_sentence(example_sentence, modifiers1, targets1)
# prettier display with IPython display
display(markup.nodes(data = True))
#print(markup.getXML())

NodeDataView({<id> 99914448661365161132906157324590123994 </id> <phrase> pneumonia </phrase> <category> ['evidence_of_pneumonia'] </category> : {'category': 'target'}})

In [5]:
# This function now works on entire documents combining all sentence-level objects into one object we can can then graph
def markup_context_document(report_text, modifiers, targets):
    context = pyConText.ConTextDocument()
    
    # we will use TextBlob for breaking up sentences
    sentences = [s.raw for s in TextBlob(report_text).sentences]
    for sentence in sentences:
        m = markup_sentence(sentence, modifiers=modifiers, targets=targets)
        context.addMarkup(m)
    
    return context

## Quesiton : Will we find a target match on this sentence? Will we match "pneumonias"?

In [6]:
example_sentence_2 = """Findings consistent with CHF, although underlying bilateral lower lobe pneumonias cannot be excluded."""

In [7]:
# let's see how things look on this sentence
markup_sentence_2 = markup_sentence(example_sentence_2, modifiers1, targets1, verbose = True)
display(markup_sentence_2.nodes(data = True))

NodeDataView({})

## We didn't mark up a target for "pneumonias" since we only had the singular variant "pneumonia"
## We can augment our targets by modifying a yaml file (.yml).  A starter yaml file is included in our course resources:
<a href="../../../edit/work/decart_rule_based_nlp/KB/pneumonia_targets.yml">KB/pneumonia_targets.yml</a>

In [8]:
import yaml
items=[]
pneumonia_targets_file = 'KB/pneumonia_targets.yml'
full_targets_path =  os.path.join(os.getcwd(), pneumonia_targets_file)
with open(full_targets_path, 'r') as f:
    for i in (yaml.safe_load_all(f)):
        items.append(i)
pd.DataFrame(items)

,Comments,Direction,Lex,Regex,Type
0,,,effusion,,EVIDENCE_OF_PNEUMONIA
1,,,pneumonia,\bpneumonia[s]?\b,EVIDENCE_OF_PNEUMONIA
2,,,infiltrate,,EVIDENCE_OF_PNEUMONIA


In [9]:
pneumonia_targets_file = 'KB/pneumonia_targets.yml'
full_targets_path =  os.path.join(os.getcwd(), pneumonia_targets_file)
full_targets_path =  os.path.join(os.getcwd(), pneumonia_targets_file)
print('Loading pneumonia;targets from : \n\t' + full_targets_path)
display(itemData.get_item_data(full_targets_path))

Loading pneumonia;targets from : 
	/home/jianlins/work/decart_rule_based_nlp/KB/pneumonia_targets.yml


[literal<<effusion>>; category<<['evidence_of_pneumonia']>>; re<<>>; rule<<>>,
 literal<<pneumonia>>; category<<['evidence_of_pneumonia']>>; re<<\bpneumonia[s]?\b>>; rule<<>>,
 literal<<infiltrate>>; category<<['evidence_of_pneumonia']>>; re<<>>; rule<<>>]

In [10]:
# Our first attempt was very simple target, so now let's add some additional concepts
targets2 = []
modifiers2 = []

pneumonia_targets_file = 'KB/pneumonia_targets.yml'

# so now let's set this up with more variants of "EVIDENCE_OF_PNEUMONIA"
full_targets_path =  os.path.join(os.getcwd(), pneumonia_targets_file)
print('Loading pneumonia;targets from : ' + full_targets_path)
targets2 = itemData.get_item_data(full_targets_path)

# let's go ahead and use this again on our updated targets
context = markup_context_document(example_document, modifiers2, targets2)
# prettier display with IPython display
display(context.getDocumentGraph().nodes(data = True))
#print(context.getXML())

Loading pneumonia;targets from : /home/jianlins/work/decart_rule_based_nlp/KB/pneumonia_targets.yml


NodeDataView({<id> 100246180146938886484983829586404969434 </id> <phrase> pneumonia </phrase> <category> ['evidence_of_pneumonia'] </category> : {'category': 'target'}, <id> 100245844219529826004192432960055544794 </id> <phrase> infiltrate </phrase> <category> ['evidence_of_pneumonia'] </category> : {'category': 'target'}})

## Let's look at this markup in HTML with colors

In [11]:
evidence_only_colors = {
    "evidence_of_pneumonia": "blue"
}

context_html = mark_document_with_html(context, colors = evidence_only_colors, default_color="black")
display(HTML(context_html))

In [12]:
view_pycontext_output(context)

## Let's also look again to see if our regular expression for "pneumonia" and "pneumonias" worked properly

In [13]:
markup_sentence_2_check = markup_context_document(example_sentence_2, modifiers2, targets2)
view_pycontext_output(markup_sentence_2_check)

## Quiz : Targets
In the sample code below, which one is the category?  

itemData(['doctor', 'PROFESSION', '(doctor|doctors)', ''])

Update the function below to pass in one of the strings below as an argument:
* 'doctor'
* 'PROFESSION'
* 'doctors'
* '' (empty string)
* '(doctor|doctors)'

In [14]:
identify_target_category('UPDATE_ME')

'INCORRECT.  Please try again.  See the documentation above for pyConText itemData ordering'

## Quiz : pyConText files
Which of the following character strings below is the delimiter (separator) for pyConText columns when reading from files by default?

* ',' (comma)
* ' ' (space)
* '\t' (tab)
* '\n' (newline)

Update the function below to pass one of these strings as its argument

In [15]:
file_delimiter_quiz('UPDATE_ME')

'INCORRECT.  Try again.  Open the file and look at it again.  What character does it look like?'

##  We've  added some pyConText targets, so let's utilize them in a classifier so that we can see that adding targets can increase our Recall even if Precision may still be low
* NOTE 1 : We will address Precision in a moment when we start working with ConText Modifiers
* NOTE 2 : You don't need to understand all of the code below.  Essentially, it predicts pneumonia and returns 1 anytime it sees at least one target "EVIDENCE_OF_PNEUMONIA"

In [16]:
class ConTextTargetOnlyClassifier(object):
    def __init__(self, modifiers, targets):
        self.modifiers = modifiers
        self.targets = targets
    def predict(self, text):
        # let's use our other functions in this notebook to perform sentence-wise markup and
        # we can then check to see if these contain any EVIDENCE_OF_PNEUMONIA category types
        context = markup_context_document(text, self.modifiers, self.targets)
        document_graph = context.getDocumentGraph()
        
        # let's walk through all of the nodes in the graph and see how many are evidence of pneumonia
        pneumonia_evidence_count = 0
        for node in document_graph.nodes():
            category_list = node.getCategory()
            for category in category_list:
                if category.upper() == 'EVIDENCE_OF_PNEUMONIA':
                    pneumonia_evidence_count += 1
            
        # do we have at least one category of pneumonia evidence here?
        return (pneumonia_evidence_count) > 0
           
# this one has only one target
classifier1 = ConTextTargetOnlyClassifier(modifiers1, targets1)
# this one has 3...
classifier2 = ConTextTargetOnlyClassifier(modifiers2, targets2)

# and now we can assess their performance
print('****************')
print('Performance for Classifier 1 : One total Target')
calculate_prediction_metrics(annotated_docs, classifier1.predict)

print('****************')
print('Performance for Classifier 2 : 3 total Targets')
calculate_prediction_metrics(annotated_docs, classifier2.predict)

****************
Performance for Classifier 1 : One total Target
Precision : 0.6896551724137931
Recall :    0.5882352941176471
F1:         0.6349206349206349

Confusion Matrix : 


Predicted,False,True
Actual,,
0,27,9
1,14,20


****************
Performance for Classifier 2 : 3 total Targets
Precision : 0.5185185185185185
Recall :    0.8235294117647058
F1:         0.6363636363636364

Confusion Matrix : 


Predicted,False,True
Actual,,
0,10,26
1,6,28


## So we have improved recall, but what are we going to do about Precision?  Since both Precision and Recall are measured equally in our F1 measure, we need to address it.  The solution to this is to improve our classification pipeline with ConText Modifiers
* Modifiers are used to modify other tags or items in the sentence.  For example, we've already added targets for evidence of pneumonia.  We can now use modifiers to treat each of these mentions as affirmed or denied (negated) for a few examples.
* Concretely if we had a sentence "No evidence of pneumonia" our current pipelines would count this as a positive case of pneumonia when it is in fact the contrary.
* We can add a rule "no evidence of" to modify all targets which follow it
* Modifiers can also modify targets which came before it in a sentence such as "pneumonia will be ruled out"

## Modifiers can have the following values:
1. backward (modify any markup preceding it)
2. forward (modify any markup following it)
3. bidirectional (modify any markup following or preceding it

## Let's look at a simple example of a modifier

In [17]:
likely_sentence = """likely represents pneumonia"""

simple_modifiers = itemData.itemData(["likely represents", "PROBABLE_EXISTENCE", "", "forward"])

context_output = markup_context_document(likely_sentence, simple_modifiers, targets2)
view_pycontext_output(context_output)

## Now let's experiment with the directionality of the modifiers (forward, backward, etc)

In [18]:
probable_sentence_1 = """probable case of pneumonia"""
probable_sentence_2 = """no evidence of pneumonia and probable arthritis"""

In [19]:
   
modifiers_forward = itemData.itemData(["probable", "PROBABLE_EXISTENCE", "", "forward"])
modifiers_backward = itemData.itemData(["probable", "PROBABLE_EXISTENCE", "", "backward"])

## Forward in this sentence modifies what we would expect

In [20]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
view_pycontext_output(markup_context_document(probable_sentence_1, modifiers_forward, targets1))


## In this example, nothing precedes "probable" so nothing to modify

In [21]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
view_pycontext_output(markup_context_document(probable_sentence_1, modifiers_backward, targets1))

## In this example, the forward rule does not pick up pneumonia since it comes before and we've not set up a target for arthritis

In [22]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
view_pycontext_output(markup_context_document(probable_sentence_2, modifiers_forward, targets1))

## And finally in this example, when we allow the target to work backward, it picks up a context where there seems to be "no evidence of pneumonia"

In [23]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
view_pycontext_output(markup_context_document(probable_sentence_2, modifiers_backward, targets1))

## Another important attribute that Modifiers can employ : "terminate"
This allows any modifier working forward or backward to stop its modifications if it encounters one of these terms.  Let's demonstrate an example where we want "probable" to modify "arthritis" as a condition but not "pneumonia":

In [24]:
terminate_example_sentence = """probable arthritis but not pneumonia"""

In [25]:
temp_targets = itemData.itemData(["pneumonia", "EVIDENCE_OF_PNEUMONIA", "", ""],
                       ["arthritis", "ANOTHER_CONDITION", "", "forward"])

modifiers_without_terminate = itemData.itemData(["probable", "PROBABLE_EXISTENCE", "", "forward"])

modifiers_with_terminate = itemData.itemData(["probable", "PROBABLE_EXISTENCE", "", "forward"],
                                   ["but", "CONJ", "", "terminate"])

## Without the "terminate" modifier, "probable_existence" is applied to both arthritis and also pneumonia

In [26]:
view_pycontext_output(markup_context_document(terminate_example_sentence, modifiers_without_terminate, temp_targets))

## Here with the "terminate" modifier "probable_existence" is applied only to "arthritis" and does not modify beyond the conjunction "but"

In [27]:
view_pycontext_output(markup_context_document(terminate_example_sentence, modifiers_with_terminate, temp_targets))

## Modifier Quiz : Directionality
Imagine that you are given the string below as a modifier.  What directionality do you think it should have as you are setting up the modifier?

'was ruled out'

Update the function below to pass in one of the following values:
* 'forward'
* 'backward'
* 'bidirectional'
* 'terminate'

In [28]:
modifier_directionality_quiz('UPDATE_ME')

'INCORRECT.  It is not clear what you passed in.  Please see the list of possible answers'

## Now let's get back to how these are targets and modifiers are used together

In [32]:
pneumonia_modifiers_file = 'KB/pneumonia_modifiers.yml'
modifier_file_path =  os.path.join(os.getcwd(), pneumonia_modifiers_file)
modifiers3 = itemData.get_item_data(modifier_file_path)
# let's just use the same targets as above for our third pipeline
targets3 = targets2


print('Total Modifiers Loaded for pipeline #3 : [{0}]'.format(len(modifiers3)))
print('Total Targets Loaded for pipeline #3 : [{0}]'.format(len(targets3)))

Total Modifiers Loaded for pipeline #3 : [8]
Total Targets Loaded for pipeline #3 : [3]


# Now we can use leverage both Targets and Modifiers to properly leverage context let's see what this looks like in HTML with our document:

In [ ]:
# prepare some colors for displaying any markup we might see
colors = {
    "evidence_of_pneumonia": "blue",
    "definite_negated_existence": "red",
    "probable_negated_existence": "indianred",
    "ambivalent_existence": "blue",
    "probable_existence": "forestgreen",
    "definite_existence": "green",
    "historical": "goldenrod",
    "indication": "pink",
    "acute": "golden"
}

# let's mark up a new context object for our pipeline#3
context3 = markup_context_document(example_document, modifiers3, targets3)

display(HTML(mark_document_with_html(context3, colors = colors, default_color="black")))

In [ ]:
# And use Brat style display:
view_pycontext_output(context3)

## OK, so now that we've got a few Targets and Modifiers to start from, let's process all of the documents and then visualize the relationships between Targets and Modifiers for some of these documents

In [ ]:
%%time
# NOTE : This is a "magic" command to Jupyter to time the execution of this entire cell

report_results = []
print('Marking up all documents...')
for anno_doc in annotated_docs:
    report_context = markup_context_document(anno_doc.text, modifiers3, targets3)
    # package this up into a class that the RadNLP utilities can use
    results = classrslts(context_document=report_context, exam_type="Chest X-Ray", report_text=anno_doc.text, classification_result='N/A')
    report_results.append(results)
    
print('DONE Marking up all documents...')

In [ ]:
# This function let's us iterate through all documents and view the markup
def view_pycontext_graph(class_results):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(class_results)-1))
    def _view_markup(i):
        class_result = class_results[i]
        view_pycontext_output(class_result.context_document)
        
        
# This function let's us iterate through all documents and view the markup
def view_annotation_markup(anno_docs, colors):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(anno_docs)-1))
    def _view_markup(i):
        report_html = pneumonia_annotation_html_markup(anno_docs[i])
        
        display(HTML(report_html))

In [ ]:
view_pycontext_graph(report_results)

## Now that we can see how modifiers can interact with Targets, this will be a useful tool to help us increase Precision by ignoring False Positives like "no evidence of pneumonia".

## Next, Jianlin will talk about a way to set up classification rules combining Targets and Modifiers

# Quiz: Frequent Modifier categories
Earlier we grouped and counted modifiers by their category.  Besides the modifiers for conjuction ('CONJ') which is the most frequent modifier category, what is the second most common modifier category in the set provided to you?

* 'PROBABLE_NEGATED_EXISTENCE'
* 'PROBABLE_EXISTENCE'
* 'DEFINITE_NEGATED_EXISTENCE'
* 'HISTORICAL'

Update the function below to pass in the most common modifier category.  Hint : You may need to scroll above to see this output

In [ ]:
second_most_frequent_modifier_quiz('UPDATE_ME')

<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2019.<br/>
Presenters : Dr. Wendy Chapman, Kelly Peterson, Alec Chapman, Jianlin Shi <br> Acknowledgement: Many thanks to Olga Patterson because part of the materials are adopted from his previous work.